In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_857176/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_6_49,0.999274,0.816471,0.997469,0.999191,0.998938,0.003169,1.227262,0.002971,0.004305,0.003638,0.104031,0.056297,1.002690,0.057198,171.508467,344.219114,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_6_48,0.999271,0.816506,0.997489,0.999217,0.998962,0.003185,1.227023,0.002947,0.004169,0.003558,0.103684,0.056438,1.002703,0.057341,171.498441,344.209088,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_6_47,0.999266,0.816543,0.997508,0.999242,0.998984,0.003203,1.226777,0.002926,0.004036,0.003481,0.103361,0.056596,1.002718,0.057501,171.487303,344.197949,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_6_46,0.999262,0.816582,0.997529,0.999267,0.999008,0.003220,1.226515,0.002901,0.003900,0.003400,0.102999,0.056749,1.002733,0.057657,171.476471,344.187117,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_6_45,0.999258,0.816622,0.997545,0.999292,0.999029,0.003241,1.226250,0.002882,0.003770,0.003326,0.102681,0.056927,1.002750,0.057838,171.463960,344.174606,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,model_1_0_4,0.894344,0.685935,0.882272,0.810740,0.860380,0.461361,2.100157,0.707929,0.565415,0.636672,0.570766,0.679236,1.391548,0.690104,161.547148,334.257795,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
454,model_1_0_3,0.892376,0.684090,0.880212,0.807942,0.858105,0.469957,2.112490,0.720315,0.573773,0.647044,0.575271,0.685534,1.398844,0.696504,161.510227,334.220873,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
455,model_1_0_2,0.890382,0.682233,0.878156,0.805152,0.855836,0.478665,2.124913,0.732674,0.582108,0.657391,0.579839,0.691856,1.406233,0.702926,161.473511,334.184157,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
456,model_1_0_1,0.888356,0.680353,0.876074,0.802290,0.853525,0.487507,2.137478,0.745196,0.590659,0.667928,0.584494,0.698217,1.413738,0.709389,161.436900,334.147546,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
